In [1]:
import pandas as pd
from lazypredict.Supervised import LazyRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [2]:
def corriger_encodage(df):
    # Fonction pour corriger l'encodage d'une chaîne de caractères
    def corriger_chaine(chaine):
        if isinstance(chaine, str):
            try:
                return chaine.encode('latin1').decode('utf-8')
            except UnicodeEncodeError:
                return chaine
        return chaine

    # Corriger les valeurs dans le DataFrame
    for col in df.columns:
        df[col] = df[col].apply(corriger_chaine)

    # Corriger les noms de colonnes
    df.columns = [corriger_chaine(col) for col in df.columns]

    return df

In [3]:
df = corriger_encodage(pd.read_csv('./data/traffic_to_ml.csv', encoding='unicode_escape').drop(columns=["Unnamed: 0"]))

In [4]:
scaler = StandardScaler()
numerical_cols = df.select_dtypes(include=['float64', 'int64']).columns

In [5]:
df.columns

Index(['Year', 'Area', 'Date', 'Road', 'Municipality Name', 'County Name',
       'Province Name', 'Fatalties', 'Serious Injuries', 'Light Injuries',
       'Total Victims', 'Units Involved', 'Pedestrains Involved',
       'Bicycles Involved', 'Mopeds Involved', 'Motorcycles',
       'Light Vehicles Involved', 'Heavy Vehicles Involved',
       'Other Units Involved', 'Unspecified Units Involved',
       'Road Speed Limit', 'Accident with Hit and Run', 'Fog Presence',
       'Surrounding Environment', 'Special Lane Presence',
       'Special Traffic Measures', 'Weather Conditions',
       'Special Road Functions', 'Severity of Accident', 'Influence of Fog',
       'Influence of Environment', 'Influence of Traffic',
       'Influence of Weather', 'Influence of Wind Intensity',
       'Influence of Lighting', 'Influence of Special Measures',
       'Influence of Road Objects', 'Influence of Road Surface',
       'Influence of Visibility', 'Intersection Characteristics',
       'Lighting C

In [6]:
X = df[numerical_cols].drop(['Total Victims'], axis=1)  # Drop non-feature columns
y = df[numerical_cols]['Total Victims']  #

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)  # Notice shuffle=False for time series

In [8]:
X_train_scaled = X_train
X_test_scaled = X_test

In [9]:
reg = LazyRegressor(verbose=0, ignore_warnings=True, custom_metric=None)
models, predictions = reg.fit(X_train_scaled, X_test_scaled, y_train, y_test)

  0%|          | 0/42 [00:00<?, ?it/s]

 98%|█████████▊| 41/42 [08:34<00:05,  5.54s/it]

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001965 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 588
[LightGBM] [Info] Number of data points in the train set: 16881, number of used features: 49
[LightGBM] [Info] Start training from score 1.549553


100%|██████████| 42/42 [08:34<00:00, 12.26s/it]


In [10]:
print(models)

                               Adjusted R-Squared  R-Squared  RMSE  Time Taken
Model                                                                         
LassoLarsIC                                  1.00       1.00  0.00        0.11
LarsCV                                       1.00       1.00  0.00        0.14
OrthogonalMatchingPursuit                    1.00       1.00  0.00        0.07
OrthogonalMatchingPursuitCV                  1.00       1.00  0.00        0.14
LassoLarsCV                                  1.00       1.00  0.00        0.13
Lars                                         1.00       1.00  0.00        0.08
LinearSVR                                    1.00       1.00  0.00        0.10
BayesianRidge                                1.00       1.00  0.00        0.10
RidgeCV                                      1.00       1.00  0.00        0.12
Ridge                                        1.00       1.00  0.00        0.06
SGDRegressor                                 1.00   